In [1]:
from pyspark import *

In [2]:
df_train = spark.read.load("/FileStore/tables/train.txt", format="csv" , sep=" ", inferSchema="true", header="false").drop('_c26','_c27')
df_test = spark.read.load("/FileStore/tables/test.txt", format="csv" , sep=" ", inferSchema="true", header="false").drop('_c26','_c27')
df_truth = spark.read.load("/FileStore/tables/truth.txt", format="csv" , sep=" ", inferSchema="true", header="false").drop('_c1')

In [3]:
df_train.show(5)

+---+---+-------+-------+-----+------+------+-------+-------+-----+-----+------+-------+-------+----+-----+------+-------+-------+------+----+----+----+-----+-----+-------+
_c0|_c1| _c2| _c3| _c4| _c5| _c6| _c7| _c8| _c9| _c10| _c11| _c12| _c13|_c14| _c15| _c16| _c17| _c18| _c19|_c20|_c21|_c22| _c23| _c24| _c25|
+---+---+-------+-------+-----+------+------+-------+-------+-----+-----+------+-------+-------+----+-----+------+-------+-------+------+----+----+----+-----+-----+-------+
 1| 1|-7.0E-4|-4.0E-4|100.0|518.67|641.82| 1589.7| 1400.6|14.62|21.61|554.36|2388.06|9046.19| 1.3|47.47|521.66|2388.02|8138.62|8.4195|0.03| 392|2388|100.0|39.06| 23.419|
 1| 2| 0.0019|-3.0E-4|100.0|518.67|642.15|1591.82|1403.14|14.62|21.61|553.75|2388.04|9044.07| 1.3|47.49|522.28|2388.07|8131.49|8.4318|0.03| 392|2388|100.0| 39.0|23.4236|
 1| 3|-0.0043| 3.0E-4|100.0|518.67|642.35|1587.99| 1404.2|14.62|21.61|554.26|2388.08|9052.94| 1.3|47.27|522.42|2388.03|8133.23|8.4178|0.03| 390|2388|100.0|38.95|23.3442|
 1| 4| 7.0E-4| 0.0|100.0|518.67|642.35|1582.79|1401.87|14.62|21.61|554.45|2388.11|9049.48| 1.3|47.13|522.86|2388.08|8133.83|8.3682|0.03| 392|2388|100.0|38.88|23.3739|
 1| 5|-0.0019|-2.0E-4|100.0|518.67|642.37|1582.85|1406.22|14.62|21.61| 554.0|2388.06|9055.15| 1.3|47.28|522.19|2388.04| 8133.8|8.4294|0.03| 393|2388|100.0| 38.9|23.4044|
+---+---+-------+-------+-----+------+------+-------+-------+-----+-----+------+-------+-------+----+-----+------+-------+-------+------+----+----+----+-----+-----+-------+
only showing top 5 rows

In [4]:
actual = [col for col in df_train.columns]
new = ['id', 'cycle', 'setting1', 'setting2', 'setting3', 's1','s2', 's3', 's4', 's5', 's6', 's7', 's8', 's9', 's10', 's11', 's12', 's13', 's14', 's15', 's16', 's17', 's18', 's19', 's20', 's21']
for i in range(len(new)):
  df_train = df_train.withColumnRenamed(actual[i],new[i])
  df_test = df_test.withColumnRenamed(actual[i],new[i])
df_truth = df_truth.withColumnRenamed('_c0','ttf')

# Reading Files

In [6]:
#train_raw = spark.read.table("pm_train_txt").drop('_c26','_c27')
#test_raw = spark.read.table("pm_test_txt").drop('_c26','_c27')
#truth_raw = spark.read.table("pm_truth_txt")
#drop_column = ['setting3','s1','s5','s10','s16','s18','s19']
#df_train = df_train.drop('setting3','s1','s5','s10','s16','s18','s19')


In [7]:
display(df_train.summary())

summary,id,cycle,setting1,setting2,setting3,s1,s2,s3,s4,s5,s6,s7,s8,s9,s10,s11,s12,s13,s14,s15,s16,s17,s18,s19,s20,s21
count,20631,20631,20631,20631,20631,20631,20631,20631,20631,20631,20631,20631,20631,20631,20631,20631,20631,20631,20631,20631,20631,20631,20631,20631,20631,20631
mean,51.506567786340945,108.80786195530997,-8.870146866366216E-6,2.350831273326574E-6,100.0,518.6699999999346,642.6809335466004,1590.5231186079263,1408.933781687751,14.619999999996605,21.609803208753878,553.3677112112824,2388.0966516407484,9065.242940720273,1.299999999999534,47.54116814502418,521.4134700208454,2388.0961523920328,8143.752722117232,8.44214558189131,0.029999999999984435,393.2106538703892,2388.0,100.0,38.81627066065643,23.289705360864783
stddev,29.22763290879931,68.88099017721827,0.002187313449015146,2.93062124566145E-4,0.0,0.0,0.5000532700605589,6.131149519690709,9.000604780543718,0.0,0.001388984912708412,0.8850922576633927,0.07098547889055705,22.082879525067696,0.0,0.26708739863969233,0.7375533922096187,0.0719189156975736,19.076175975950626,0.03750503795196585,0.0,1.5487630246142872,0.0,0.0,0.18074642787365824,0.10825087474491285
min,1,1,-0.0087,-6.0E-4,100.0,518.67,641.21,1571.04,1382.25,14.62,21.6,549.85,2387.9,9021.73,1.3,46.85,518.69,2387.88,8099.94,8.3249,0.03,388,2388,100.0,38.14,22.8942
25%,26,52,-0.0015,-2.0E-4,100.0,518.67,642.32,1586.26,1402.36,14.62,21.61,552.81,2388.05,9053.1,1.3,47.35,520.96,2388.04,8133.24,8.4148,0.03,392,2388,100.0,38.7,23.2217
50%,52,104,0.0,0.0,100.0,518.67,642.64,1590.1,1408.03,14.62,21.61,553.44,2388.09,9060.66,1.3,47.51,521.48,2388.09,8140.54,8.4389,0.03,393,2388,100.0,38.83,23.2979
75%,77,156,0.0015,3.0E-4,100.0,518.67,643.0,1594.38,1414.55,14.62,21.61,554.01,2388.14,9069.42,1.3,47.7,521.95,2388.14,8148.3,8.4655,0.03,394,2388,100.0,38.95,23.3668
max,100,362,0.0087,6.0E-4,100.0,518.67,644.53,1616.91,1441.49,14.62,21.61,556.06,2388.56,9244.59,1.3,48.53,523.38,2388.56,8293.72,8.5848,0.03,400,2388,100.0,39.43,23.6184


# Feature engineering Creating class for regression & Classification models
## ttf = Time to failure
    Binary classification 
    if ttf<30 
        fast failure 
    else 
      accordingly based on binary and multi classification

In [9]:
maximum = df_train.groupby('id').agg({'cycle':'max'}).withColumnRenamed('id','Ids').withColumnRenamed('max(cycle)','Max')
#maximum.show(100)
step1 = df_train.join(maximum, df_train.id == maximum.Ids,how='left').drop('Ids')# Join 'left_outer'
#step1.show(5)
#genrate time to failure after how many stages = max cycle - cycle
step2 = step1.withColumn("ttf", step1.Max-step1.cycle).drop('Max')
#step2.show(5)

# Feature transformations
# Pipeline Stages
## (stage 1) Bucketizing 
## (stage 2) Min-Max scaler 
## (stage 3) vector assembler 
## (stage 4) Dataset for classification and Regression

In [11]:
from pyspark.ml.feature import Bucketizer
from pyspark.ml import Pipeline
bucketizer = [Bucketizer(splits=[-float("inf"),30,float("inf")], inputCol="ttf", outputCol= "binary")]
bucketizer += [Bucketizer(splits=[-float("inf"),15,30,60,float("inf")], inputCol="ttf", outputCol= "ternary")]
stage = bucketizer
step3 = Pipeline(stages=stage).fit(step2).transform(step2)

# scaling my columns between 0 & 1

In [13]:
from pyspark.ml.feature import MinMaxScaler, VectorAssembler
assembler = [VectorAssembler(inputCols= [col], outputCol="Vect_"+col)  for col in step3.columns if col not in ['id','cycle','ttf','binary','ternary']]
minmax = [MinMaxScaler(inputCol="Vect_"+col, outputCol='scaled_'+col) for col in step3.columns if col not in ['id','cycle','ttf','binary','ternary']]
stage = assembler + minmax
step4 = Pipeline(stages = stage).fit(step3).transform(step3).select(*['scaled_'+col for col in step3.columns if col not in ['id','cycle','ttf','binary','ternary']]+['id','cycle','ttf','binary','ternary'])

In [14]:
# UDF for converting column type from vector to double type
from pyspark.sql.types import DoubleType
unlist = udf(lambda x: round(float(list(x)[0]),3), DoubleType())

for col, types in step4.dtypes:
  if types =='vector':
    step4 = step4.withColumn(col, unlist(col))

In [15]:
features = VectorAssembler(inputCols= [col for col in step4.columns if col not in ['ttf','binary','ternary']], outputCol="features")  
step5= features.transform(step4)

In [16]:
from pyspark.ml.feature import Bucketizer
from pyspark.ml import Pipeline
bucketizer = [Bucketizer(splits=[-float("inf"),30,float("inf")], inputCol="ttf", outputCol= "binary")]
bucketizer += [Bucketizer(splits=[-float("inf"),15,30,60,float("inf")], inputCol="ttf", outputCol= "ternary")]

assembler = [VectorAssembler(inputCols= [col], outputCol="Vect_"+col)  for col in step2.columns if col not in ['id','cycle','ttf']]
minmax = [MinMaxScaler(inputCol="Vect_"+col, outputCol='scaled_'+col) for col in step2.columns if col not in ['id','cycle','ttf']]
assembler1 = VectorAssembler(inputCols= ['scaled_'+col for col in step2.columns if col not in ['ttf','id','cycle']]+['id','cycle'], outputCol="features")  
stage = bucketizer + assembler + minmax+ [assembler1]
stepf = Pipeline(stages = stage).fit(step2).transform(step2).select(*['scaled_'+col for col in step2.columns if col not in ['id','cycle','ttf','binary','ternary']]+['id','cycle','features','ttf','binary','ternary'])

In [17]:
#dataset is ready for Machine Learning
display(stepf)

scaled_setting1,scaled_setting2,scaled_setting3,scaled_s1,scaled_s2,scaled_s3,scaled_s4,scaled_s5,scaled_s6,scaled_s7,scaled_s8,scaled_s9,scaled_s10,scaled_s11,scaled_s12,scaled_s13,scaled_s14,scaled_s15,scaled_s16,scaled_s17,scaled_s18,scaled_s19,scaled_s20,scaled_s21,id,cycle,features,ttf,binary,ternary
"List(1, 1, List(), List(0.4597701149425288))","List(1, 1, List(), List(0.16666666666666663))","List(1, 1, List(), List(0.5))","List(1, 1, List(), List(0.5))","List(1, 1, List(), List(0.18373493975904379))","List(1, 1, List(), List(0.40680183126226366))","List(1, 1, List(), List(0.30975692099932317))","List(1, 1, List(), List(0.5))","List(1, 1, List(), List(1.0))","List(1, 1, List(), List(0.7262479871175599))","List(1, 1, List(), List(0.2424242424240754))","List(1, 1, List(), List(0.10975500314098934))","List(1, 1, List(), List(0.5))","List(1, 1, List(), List(0.36904761904761757))","List(1, 1, List(), List(0.6332622601279213))","List(1, 1, List(), List(0.2058823529410388))","List(1, 1, List(), List(0.19960780266281528))","List(1, 1, List(), List(0.36398614851866024))","List(1, 1, List(), List(0.5))","List(1, 1, List(), List(0.3333333333333333))","List(1, 1, List(), List(0.5))","List(1, 1, List(), List(0.5))","List(1, 1, List(), List(0.7131782945736452))","List(1, 1, List(), List(0.7246616956641801))",1,1,"List(1, 26, List(), List(0.4597701149425288, 0.16666666666666663, 0.5, 0.5, 0.18373493975904379, 0.40680183126226366, 0.30975692099932317, 0.5, 1.0, 0.7262479871175599, 0.2424242424240754, 0.10975500314098934, 0.5, 0.36904761904761757, 0.6332622601279213, 0.2058823529410388, 0.19960780266281528, 0.36398614851866024, 0.5, 0.3333333333333333, 0.5, 0.5, 0.7131782945736452, 0.7246616956641801, 1.0, 1.0))",191,1.0,3.0
"List(1, 1, List(), List(0.6091954022988506))","List(1, 1, List(), List(0.25))","List(1, 1, List(), List(0.5))","List(1, 1, List(), List(0.5))","List(1, 1, List(), List(0.28313253012046957))","List(1, 1, List(), List(0.45301940265968865))","List(1, 1, List(), List(0.35263335584064986))","List(1, 1, List(), List(0.5))","List(1, 1, List(), List(1.0))","List(1, 1, List(), List(0.6280193236715017))","List(1, 1, List(), List(0.21212121212106597))","List(1, 1, List(), List(0.10024230458583903))","List(1, 1, List(), List(0.5))","List(1, 1, List(), List(0.3809523809523814))","List(1, 1, List(), List(0.7654584221748323))","List(1, 1, List(), List(0.27941176470602985))","List(1, 1, List(), List(0.16281349984518642))","List(1, 1, List(), List(0.41131204309350256))","List(1, 1, List(), List(0.5))","List(1, 1, List(), List(0.3333333333333333))","List(1, 1, List(), List(0.5))","List(1, 1, List(), List(0.5))","List(1, 1, List(), List(0.6666666666666666))","List(1, 1, List(), List(0.7310135321734316))",1,2,"List(1, 26, List(), List(0.6091954022988506, 0.25, 0.5, 0.5, 0.28313253012046957, 0.45301940265968865, 0.35263335584064986, 0.5, 1.0, 0.6280193236715017, 0.21212121212106597, 0.10024230458583903, 0.5, 0.3809523809523814, 0.7654584221748323, 0.27941176470602985, 0.16281349984518642, 0.41131204309350256, 0.5, 0.3333333333333333, 0.5, 0.5, 0.6666666666666666, 0.7310135321734316, 1.0, 2.0))",190,1.0,3.0
"List(1, 1, List(), List(0.2528735632183908))","List(1, 1, List(), List(0.75))","List(1, 1, List(), List(0.5))","List(1, 1, List(), List(0.5))","List(1, 1, List(), List(0.3433734939759061))","List(1, 1, List(), List(0.36952256376716813))","List(1, 1, List(), List(0.3705266711681304))","List(1, 1, List(), List(0.5))","List(1, 1, List(), List(1.0))","List(1, 1, List(), List(0.7101449275362356))","List(1, 1, List(), List(0.2727272727270848))","List(1, 1, List(), List(0.14004307637081964))","List(1, 1, List(), List(0.5))","List(1, 1, List(), List(0.25000000000000105))","List(1, 1, List(), List(0.7953091684434864))","List(1, 1, List(), List(0.2205882352943045))","List(1, 1, List(), List(0.17179275467024466))","List(1, 1, List(), List(0.35744517121970065))","List(1, 1, List(), List(0.5))","List(1, 1, List(), List(0.16666666666666666))","Li

# Correlation Matrix

In [19]:
from pyspark.mllib.stat import Statistics
import pandas as pd
import seaborn as sns

# df = sqlCtx.read.format('com.databricks.spark.csv').option('header', 'true').option('inferschema', 'true').load('corr_test.csv')
df = step2
col_names = df.columns
features = df.rdd.map(lambda row: row[0:])
corr_mat=Statistics.corr(features, method="pearson",)
corr_df = pd.DataFrame(corr_mat.round(4))
corr_df.index, corr_df.columns = col_names, col_names
corr_df = corr_df.reset_index()
display(corr_df)

index,id,cycle,setting1,setting2,setting3,s1,s2,s3,s4,s5,s6,s7,s8,s9,s10,s11,s12,s13,s14,s15,s16,s17,s18,s19,s20,s21,ttf
id,1.0,0.0788,-0.0179,-0.0062,null,-0.0,0.0141,0.0126,0.0261,-0.0,0.0264,-0.0321,0.0399,-0.0521,null,0.0247,-0.0319,0.0442,-0.0595,0.0225,null,0.0135,0.0,null,-0.0206,-0.0165,0.0788
cycle,0.0788,1.0,-0.0045,0.0161,null,-0.0,0.5499,0.5439,0.6246,-0.0,0.106,-0.5959,0.476,0.444,null,0.6344,-0.6114,0.4775,0.3703,0.5887,null,0.567,0.0,null,-0.5836,-0.5859,-0.7362
setting1,-0.0179,-0.0045,1.0,0.0117,null,-0.0,0.009,-0.0057,0.0095,-0.0,-0.0013,-0.0094,-4.0E-4,-0.0043,null,0.0117,-0.0015,0.0023,-0.0045,0.0077,null,0.0026,-0.0,null,-0.0057,-0.0146,-0.0032
setting2,-0.0062,0.0161,0.0117,1.0,null,0.0,0.0073,0.0091,0.0147,-0.0,0.0144,-0.0167,0.0133,-0.0054,null,0.0117,-0.0109,0.0182,-0.0063,0.0142,null,0.0123,-0.0,null,-0.0106,-0.0078,-0.0019
setting3,null,null,null,null,1.0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
s1,-0.0,-0.0,-0.0,0.0,null,1.0,-0.0,-0.0,0.0,0.1092,0.0032,-0.0,0.0026,0.0,null,-0.0,-0.0,0.0013,0.0,-0.0,null,-0.0,-47.1606,null,-0.0,0.0,0.0
s2,0.0141,0.5499,0.009,0.0073,null,-0.0,1.0,0.6026,0.7149,-1.0E-4,0.1322,-0.7021,0.6623,0.2738,null,0.74,-0.7246,0.6618,0.1798,0.676,null,0.6299,-3.0E-4,null,-0.6618,-0.6681,-0.6065
s3,0.0126,0.5439,-0.0057,0.0091,null,-0.0,0.6026,1.0,0.6784,-0.0,0.116,-0.6646,0.6025,0.323,null,0.6959,-0.6803,0.601,0.2371,0.6399,null,0.6,0.0,null,-0.6259,-0.6339,-0.5845
s4,0.0261,0.6246,0.0095,0.0147,null,0.0,0.7149,0.6784,1.0,-0.0,0.1505,-0.7931,0.7468,0.2974,null,0.8301,-0.8156,0.7452,0.1907,0.7585,null,0.7035,-0.0,null,-0.7481,-0.7452,-0.6789
s5,-0.0,-0.0,-0.0,-0.0,null,0.1092,-1.0E-4,-0.0,-0.0,1.0,-0.0021,-0.0,0.002,-0.0,null,-0.0,0.0,0.0014,0.0,-0.0,null,0.0,-30.8542,null,-0.0,-0.0,0.0


# Three Dataset
## 1. Regression dataset with time to failure as class
## 2. Binary classification using 'binary' class
## 3. Multi classification using 'ternanry' class

In [21]:
train1 = stepf.select('features','ttf').withColumnRenamed('ttf','label')
train2 = stepf.select('features','binary').withColumnRenamed('binary','label')
train3 = stepf.select('features','ternary').withColumnRenamed('ternary','label')

# Test Data processing

In [24]:
import pyspark.sql.functions as sparkf
#df_test = df_test.drop('setting3','s1','s5','s10','s16','s18','s19')
truth = df_truth.withColumn('Ids', sparkf.monotonically_increasing_id()+1).select('Ids','ttf')
maximum = df_test.groupby('id').agg({'cycle':'max'}).withColumnRenamed('id','Ids').withColumnRenamed('max(cycle)','Max')
step1 = df_test.join(maximum, df_test.id == maximum.Ids,how='left').drop('Ids')# Join 'left_outer'
step1 = step1[step1['cycle'] == step1['Max']].drop('Max')
test = step1.join(truth, step1.id == truth.Ids,how='left').drop('Ids')# Join 'left_outer'
test.show()


+---+-----+--------+--------+--------+------+------+-------+-------+-----+-----+------+-------+-------+---+-----+------+-------+-------+------+----+---+----+-----+-----+-------+---+
 id|cycle|setting1|setting2|setting3| s1| s2| s3| s4| s5| s6| s7| s8| s9|s10| s11| s12| s13| s14| s15| s16|s17| s18| s19| s20| s21|ttf|
+---+-----+--------+--------+--------+------+------+-------+-------+-----+-----+------+-------+-------+---+-----+------+-------+-------+------+----+---+----+-----+-----+-------+---+
 1| 31| -6.0E-4| 4.0E-4| 100.0|518.67|642.58|1581.22|1398.91|14.62|21.61|554.42|2388.08| 9056.4|1.3|47.23|521.79|2388.06|8130.11|8.4024|0.03|393|2388|100.0|38.81|23.3552|112|
 2| 49| 0.0018| -1.0E-4| 100.0|518.67|642.55|1586.59|1410.83|14.62|21.61|553.52| 2388.1|9044.77|1.3|47.67|521.74|2388.09| 8126.9|8.4505|0.03|391|2388|100.0|38.81|23.2618| 98|
 3| 126| -0.0016| 4.0E-4| 100.0|518.67|642.88|1589.75|1418.89|14.62|21.61|552.59|2388.16|9049.26|1.3|47.88|520.83|2388.14|8131.46|8.4119|0.03|395|2388|100.0|38.93| 23.274| 69|
 4| 106| 0.0012| 4.0E-4| 100.0|518.67|642.78|1594.53|1406.88|14.62|21.61|552.64|2388.13| 9051.3|1.3|47.65|521.88|2388.11|8133.64|8.4634|0.03|395|2388|100.0|38.58|23.2581| 82|
 5| 98| -0.0013| -4.0E-4| 100.0|518.67|642.27|1589.94|1419.36|14.62|21.61|553.29| 2388.1|9053.99|1.3|47.46| 521.0|2388.15|8125.74|8.4362|0.03|394|2388|100.0|38.75|23.4117| 91|
 6| 105| 0.0076| -3.0E-4| 100.0|518.67|643.05|1586.94|1404.49|14.62|21.61|553.73|2388.14|9055.83|1.3|47.51|521.56|2388.15|8139.02|8.4452|0.03|393|2388|100.0|38.91|23.3269| 93|
 7| 160| 0.0016| -1.0E-4| 100.0|518.67| 642.1|1589.59|1413.57|14.62|21.61|553.42|2388.04|9070.99|1.3|47.32|522.32|2388.07|8149.19|8.4028|0.03|393|2388|100.0|38.91|23.2763| 91|
 8| 166| 0.0016| -5.0E-4| 100.0|518.67|642.59|1591.91|1413.89|14.62|21.61|553.51|2388.14|9049.96|1.3|47.59|520.53|2388.12|8127.84|8.4744|0.03|393|2388|100.0|38.73|23.2465| 95|
 9| 55| -3.0E-4| 4.0E-4| 100.0|518.67|642.27|1593.61|1410.27|14.62|21.61|552.58|2388.13|9057.57|1.3|47.58|521.65|2388.16|8134.76|8.4293|0.03|392|2388|100.0|38.78|23.3515|111|
 10| 192| -0.0018| 4.0E-4| 100.0|518.67| 643.0| 1589.5|1398.99|14.62|21.61|552.88|2388.11|9060.88|1.3|47.59|521.82|2388.08|8141.91|8.4194|0.03|393|2388|100.0|38.79|23.2956| 96|
 11| 83| 1.0E-4| 4.0E-4| 100.0|518.67|642.42|1587.73|1414.22|14.62|21.61| 552.8|2388.21|9037.11|1.3|47.67|520.74|2388.18|8116.85| 8.464|0.03|394|2388|100.0|38.66|23.2908| 97|
 12| 217| 0.0013| 3.0E-4| 100.0|518.67|642.59| 1595.9|1409.64|14.62|21.61|553.36|2388.04|9074.24|1.3|47.37|522.03|2388.02|8155.74|8.4482|0.03|393|2388|100.0|38.81|23.2861|124|
 13| 195| 0.0032| -4.0E-4| 100.0|518.67|642.37|1591.31|1414.51|14.62|21.61|553.28| 2388.1|9060.92|1.3|47.52| 521.3|2388.12| 8146.8|8.4336|0.03|394|2388|100.0|38.75| 23.282| 95|
 14| 46| 0.0013| -2.0E-4| 100.0|518.67|642.91|1591.39|1408.35|14.62|21.61|553.75|2388.17|9037.35|1.3|47.57|520.66| 2388.1| 8118.1|8.4233|0.03|393|2388|100.0|38.69|23.2603|107|
 15| 76| -0.0012| -0.0| 100.0|518.67|642.49|1589.21|1400.61|14.62|21.61|554.08| 2388.1| 9056.1|1.3|47.25|522.12|2388.11|8139.75|8.4067|0.03|392|2388|100.0|39.17|23.4085| 83|
 16| 113| -5.0E-4| 2.0E-4| 100.0|518.67|642.99|1587.03| 1401.0|14.62|21.61| 554.6|2388.08|9059.75|1.3|47.37|521.77|2388.08| 8147.4|8.4299|0.03|391|2388|100.0|38.81| 23.304| 84|
 17| 165| 9.0E-4| -1.0E-4| 100.0|518.67| 642.9|1599.17|1412.54|14.62|21.61|553.49|2388.06|9094.72|1.3|47.55|521.52|2388.03|8171.98|8.4319|0.03|394|2388|100.0| 38.7|23.2948| 50|
 18| 133| 7.0E-4| 1.0E-4| 100.0|518.67| 643.0|1600.45|1419.18|14.62|21.61|553.36|2388.14|9056.09|1.3| 47.7|521.06|2388.19| 8136.8|8.4418|0.03|395|2388|100.0|38.71|23.2637| 28|
 19| 135| 4.0E-4| 3.0E-4| 100.0|518.67|642.79|1588.88|1406.32|14.62|21.61|553.87| 2388.1|9055.54|1.3|47.35|521.48|2388.06| 8140.2|8.4768|0.03|393|2388|100.0|38.83|23.3297| 87|
 20| 184| -6.0E-4| -4.0E-4| 100.0|518.67|643.75|1595.12| 1419.6|14.62|21.61|552.19|2388.14|9066.49|1.3|48.13|520.64|2388.21|8138.86| 8.469|0.03|395|2388|100.0|

In [25]:
stept = Pipeline(stages = stage).fit(test).transform(test).select(*['scaled_'+col for col in test.columns if col not in ['id','cycle','ttf','binary','ternary']]+['id','cycle','features','ttf','binary','ternary'])

In [26]:
test1 = stept.select('features','ttf').withColumnRenamed('ttf','label')
test2 = stept.select('features','binary').withColumnRenamed('binary','label')
test3 = stept.select('features','ternary').withColumnRenamed('ternary','label')

# Regression Model

## Model 1: Linear Regression

In [28]:
from pyspark.ml.regression import LinearRegression
lr = LinearRegression(featuresCol = 'features', labelCol = 'label')

lr_model = lr.fit(train1)

lr_pred_train = lr_model.transform(train1)
pred.show(5)

#check on test data
lr_pred_test = lr_model.transform(test1)
predt.show(5)


from pyspark.ml.evaluation import RegressionEvaluator 
lr_evaluator = RegressionEvaluator(predictionCol='prediction', labelCol='label')
print(lr_evaluator.setMetricName('rmse').evaluate(lr_pred_train))
print(lr_evaluator.setMetricName('rmse').evaluate(lr_pred_test))


+--------------------+-----+------------------+
 features|label| prediction|
+--------------------+-----+------------------+
[0.45977011494252...| 191|167.30980593018302|
[0.60919540229885...| 190|163.32359684642284|
[0.25287356321839...| 189| 171.0038382880454|
[0.54022988505747...| 188|179.53373362778623|
[0.39080459770114...| 187| 163.6053660185492|
+--------------------+-----+------------------+
only showing top 5 rows

+--------------------+-----+------------------+
 features|label| prediction|
+--------------------+-----+------------------+
[0.39259259259259...| 112|168.13521035766757|
[0.57037037037037...| 98|121.48753128443451|
[0.31851851851851...| 69| 53.94355789905262|
[0.52592592592592...| 82| 76.77911650264996|
[0.34074074074074...| 91| 92.56317936199805|
+--------------------+-----+------------------+
only showing top 5 rows

39.57671741182032
36.5600281987422

In [29]:
# Cross Validation Model

In [30]:
# estimator
lr = LinearRegression(featuresCol = 'features', labelCol = 'label')

# parameter grid
from pyspark.ml.tuning import ParamGridBuilder
param_grid = ParamGridBuilder().\
    addGrid(lr.regParam, [0, 0.5, 1]).\
    addGrid(lr.elasticNetParam, [0, 0.5, 1]).\
    build()
    
# evaluator
evaluator = RegressionEvaluator(predictionCol='prediction', labelCol='label', metricName='rmse')

# cross-validation model
from pyspark.ml.tuning import CrossValidator
cv = CrossValidator(estimator=lr, estimatorParamMaps=param_grid, evaluator=evaluator, numFolds=4)

cv_model = cv.fit(train1)

pred_training_cv = cv_model.transform(train1)
pred_test_cv = cv_model.transform(test1)


/databricks/spark/python/pyspark/ml/util.py:791: UserWarning: Can not find mlflow. To enable mlflow logging, install MLflow library from PyPi.
 warnings.warn(_MLflowInstrumentation._NO_MLFLOW_WARNING)

In [31]:
# performance on training data
evaluator.setMetricName('rmse').evaluate(pred_training_cv)

Out[92]: 39.577783348262834

In [32]:
# performance on test data
evaluator.setMetricName('rmse').evaluate(pred_test_cv)

Out[93]: 36.555946329141754

## Model 2: Random Forest

In [34]:
from pyspark.ml import Pipeline
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml.feature import VectorIndexer
from pyspark.ml.evaluation import RegressionEvaluator

# Train a RandomForest model.
rf = RandomForestRegressor(featuresCol="features")


# Train model.  This also runs the indexer.
model = rf.fit(train1)
#checking model
rf_pred_train = model.transform(train1)

actual.select("prediction", "label").show(5)

# Make predictions.
rf_pred_test = model.transform(test1)

# Select example rows to display.
predictions.select("prediction", "label").show(5)

# Select (prediction, true label) and compute test error
rf_evaluator = RegressionEvaluator(labelCol="label", predictionCol="prediction", metricName="rmse")

rmse1 = rf_evaluator.evaluate(rf_pred_train)
print("Root Mean Squared Error (RMSE) on train data = %g" % rmse1)

rmse = rf_evaluator.evaluate(rf_pred_test)
print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)

+------------------+-----+
 prediction|label|
+------------------+-----+
166.43275626181907| 191|
172.33731190699055| 190|
174.92182459634134| 189|
185.91001015537128| 188|
170.79997171857286| 187|
+------------------+-----+
only showing top 5 rows

+------------------+-----+
 prediction|label|
+------------------+-----+
178.73131198644302| 112|
141.68852630283442| 98|
 38.02229023503855| 69|
 81.74239535626924| 82|
 78.09205445117448| 91|
+------------------+-----+
only showing top 5 rows

Root Mean Squared Error (RMSE) on train data = 35.7447
Root Mean Squared Error (RMSE) on test data = 27.4119

## Model 3: Gradient Boosting DT

In [36]:
from pyspark.ml.regression import GBTRegressor
gbt = GBTRegressor(featuresCol = 'features', labelCol = 'label', maxIter=10)
gbt_model = gbt.fit(train1)

gbt_pred_train = gbt_model.transform(train1)
gbt_pred_train.select('prediction', 'label', 'features').show(5)

gbt_pred_test = gbt_model.transform(test1)
gbt_pred_train.select('prediction', 'label', 'features').show(5)

+------------------+-----+--------------------+
 prediction|label| features|
+------------------+-----+--------------------+
180.88199992032546| 191|[0.45977011494252...|
 183.1005356111| 190|[0.60919540229885...|
181.01002265514106| 189|[0.25287356321839...|
 210.395650450927| 188|[0.54022988505747...|
179.76372843187102| 187|[0.39080459770114...|
+------------------+-----+--------------------+
only showing top 5 rows

+------------------+-----+--------------------+
 prediction|label| features|
+------------------+-----+--------------------+
180.88199992032546| 191|[0.45977011494252...|
 183.1005356111| 190|[0.60919540229885...|
181.01002265514106| 189|[0.25287356321839...|
 210.395650450927| 188|[0.54022988505747...|
179.76372843187102| 187|[0.39080459770114...|
+------------------+-----+--------------------+
only showing top 5 rows

In [37]:
gbt_evaluator = RegressionEvaluator(labelCol="label", predictionCol="prediction", metricName="rmse")

rmse = gbt_evaluator.evaluate(gbt_pred_train)
print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)


rmse = gbt_evaluator.evaluate(gbt_pred_test)
print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)

Root Mean Squared Error (RMSE) on test data = 29.8719
Root Mean Squared Error (RMSE) on test data = 38.8466

In [38]:
from pyspark.sql.types import *
schema = StructType([StructField('model', StringType(), False),StructField('train rmse', DoubleType(), True),StructField('test rmse', DoubleType(), True)])
df = sqlContext.createDataFrame(sc.emptyRDD(), schema)

list = ['lr', 'rf', 'gbt']
columns = df.columns
for model in list:
  evaluators = eval(model+"_evaluator")
  training_set = eval(model+"_pred_train")
  test_set = eval(model+"_pred_test")
  trn = evaluators.evaluate(training_set)
  tst = evaluators.evaluate(test_set)
  
  newRow = spark.createDataFrame([(model,trn,tst)], columns)
  df = df.union(newRow)
  #appended.show()
  

In [39]:
df.show()

+-----+-----------------+------------------+
model| train| test|
+-----+-----------------+------------------+
 lr|39.57671741182032| 36.5600281987422|
 rf|35.74470326516795|27.411942754104892|
 gbt|29.87186721643461| 38.84659908302241|
+-----+-----------------+------------------+